In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import datetime
import pickle
import os.path
import base64
import pandas as pd
from io import BytesIO
import os
import openpyxl
import mysql.connector
from sshtunnel import SSHTunnelForwarder
import pymysql
import mysql.connector
from requests.structures import CaseInsensitiveDict
import requests

# If modifying these SCOPES, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


#Function to handle authentication and authorization to access the Gmail API using OAuth 2.0
def get_gmail_service():
    creds = None
    # The file token.pickle stores the user's access and refresh tokens.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '/Users/germankosenkov/Code projects/Crawling/3. Crawling Emails/New/client_secret_811893502700-tj4c2bv2942q1ua3459au95sc1ohoqle.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    return service

#Function to search for a message using gmail address, attachment contained in the email etc. 
def search_messages(service, query):
    result = service.users().messages().list(userId='me', q=query).execute()
    messages = []
    if 'messages' in result:
        messages.extend(result['messages'])
    return messages


#Function to download a specific attachment from a Gmail message and saves it to a specified directory
def get_attachments(service, message_id, store_dir, desired_filename):
    try:
        message = service.users().messages().get(userId='me', id=message_id).execute()
        parts = message.get('payload', {}).get('parts', [])
        for part in parts:
            if part['filename'] == desired_filename:  # Check if the filename matches the desired one
                if 'data' in part['body']:
                    data = part['body']['data']
                else:
                    att_id = part['body'].get('attachmentId')
                    att = service.users().messages().attachments().get(userId='me', messageId=message_id, id=att_id).execute()
                    data = att['data']

                file_data = base64.urlsafe_b64decode(data.encode('UTF-8'))
                path = os.path.join(store_dir, part['filename'])

                if not os.path.exists(store_dir):
                    os.makedirs(store_dir)
                with open(path, 'wb') as f:
                    f.write(file_data)
    except Exception as e:
        print(f"An error occurred: {e}")


current_date = datetime.date.today()
current_week = current_date.isocalendar()[1]
previous_week = current_week - 1
current_year = datetime.date.today().year

new_file_path = ''


def main():
    service = get_gmail_service()
    search_query = 'from:XXXX@hotmail.com has:attachment filename:XXXX.xlsx'
    messages = search_messages(service, search_query)

    if messages:
        latest_message = messages[0]  # Get the latest email

        # Construct the path for saving the attachment with the week and year in the file name
        #folder_name = f'{current_year}_Week_{previous_week}'
        save_path = '/Users/germankosenkov/Code projects/Crawling/3. Crawling Emails/New/TRK_imports'
        

        # Call get_attachments with the desired filename
        get_attachments(service, latest_message['id'], save_path, 'XXXX.xlsx')

        # Path to the downloaded Excel file
        old_file_path = os.path.join(save_path, 'XXXX.xlsx')
        new_file_path = os.path.join(save_path, f'{current_year}_Week_{previous_week}.xlsx')

        try:
            # Rename the downloaded file
            os.rename(old_file_path, new_file_path)
            return new_file_path

            # excel_data = pd.read_excel(new_file_path)
            # print(excel_data)
        except Exception as e:
            print(f"Error reading or renaming the Excel file: {e}")

if __name__ == '__main__':
    new_file_path = main()
    print(f"New file path: {new_file_path}")


New file path: /Users/germankosenkov/Code projects/Crawling/3. Crawling Emails/New/TRK_imports/2024_Week_8.xlsx


In [2]:

from Constants import dates, countries

#Connect to the database into which to insert the data from the crawled excel file using SSHTunnelForwarder
b_ssh_host = 'XXXX'
b_ssh_user = 'XXXX'
b_ssh_port = 'XXXX'
b_ssh_private_key = 'XXXX'
b_sql_hostname = 'XXXX'
b_sql_username = 'XXXX'
b_sql_password = 'XXXX'
b_sql_database = 'XXXX'
b_sql_port = 'XXXX'

with SSHTunnelForwarder(
        (b_ssh_host, b_ssh_port),
        ssh_username=b_ssh_user,
        ssh_pkey=b_ssh_private_key,
        remote_bind_address=(b_sql_hostname, b_sql_port)) as tunnel:

    b_conn = mysql.connector.connect(
        host='127.0.0.1',
        user=b_sql_username,
        passwd=b_sql_password,
        db=b_sql_database,
        port=tunnel.local_bind_port
    )

    b_cursor = b_conn.cursor()

    # Path to the Excel file
    workbook = openpyxl.load_workbook(new_file_path, data_only=True)
    # Select the sheet to extract data from
    sheet = workbook['TOPLAM']  # You can also specify the sheet name like this: sheet = workbook['Sheet1']

    # Iterate through the rows and extract data from columns month,quantity,country_of_origin
    for row in sheet.iter_rows(min_row=2, values_only=True):
        # Assuming columns A, B, and C correspond to index 0, 1, and 2
        month = row[5]#changed
        quantity = row[6]
        country_of_origin = row[7]


        # Check if the values are not headers and not None
        if month and quantity and country_of_origin:
            if month != "Ay" and quantity != "Miktar" and country_of_origin != "Ülke":
                # Find the corresponding date and country values from dictionaries
                for date_dict in dates:
                    for key, value in date_dict.items():
                        if key == month:
                            month = value
                            break

                for country_dict in countries:
                    for key, value in country_dict.items():
                        if key == country_of_origin:
                            country_of_origin = value
                            break

                #Before inserting check if the data already exists in the table
                #If exists we will update the row as in the new coming files data can be revised by the source
                #if data doesn't exist we insert a new row 
                check_data_query = "SELECT * FROM TradeData WHERE product_id = %s AND date = %s AND raw_trade_quantity = %s AND origin_country_id = %s AND destination_country_id_raw = %s AND data_source_id = %s"
                val = (859, month, quantity, country_of_origin, 180, 4)
                b_cursor.execute(check_data_query, val)
                existing_data = b_cursor.fetchone() 

                try:
                    b_conn.autocommit = False

                    if existing_data:
                        print("Replacing existing data for", month)
                        update_query = "UPDATE TradeData SET raw_trade_quantity = %s, destination_country_id_raw = %s, origin_country_id = %s, trade_quantity = %s, raw_unit = %s, unit = %s, frequency = %s WHERE product_id = %s AND date = %s AND data_source_id = %s AND destination_country_id_raw = %s AND origin_country_id = %s"
                        update_values = (quantity, 180, country_of_origin, quantity, 'mt', 'mt', 'monthly', 859, month, 4, 180, country_of_origin)
                        b_cursor.execute(update_query, update_values)
                    else:
                        print("Inserting data for", month)
                        insert_query = "INSERT INTO TradeData (product_id, date, raw_trade_quantity, destination_country_id_raw, origin_country_id, trade_quantity, raw_unit, unit, data_source_id, frequency) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                        insert_values = (859, month, quantity, 180, country_of_origin, quantity, 'mt', 'mt', 4, 'monthly')
                        b_cursor.execute(insert_query, insert_values)

                    b_conn.commit()  # Commit the transaction
                
                except Exception as e:
                    b_conn.rollback()  # Rollback the transaction if an error occurs
                    print("Error occurred:", e)

                finally:
                    b_conn.autocommit = True


    b_cursor.close()
    b_conn.close()

print(b_cursor.rowcount, "record(s) inserted.")



Replacing existing data for 2023-07-31
Replacing existing data for 2023-07-31
Replacing existing data for 2023-07-31
Replacing existing data for 2023-07-31
Replacing existing data for 2023-08-31
Replacing existing data for 2023-08-31
Replacing existing data for 2023-08-31
Replacing existing data for 2023-08-31
Replacing existing data for 2023-08-31
Replacing existing data for 2023-09-30
Replacing existing data for 2023-09-30
Replacing existing data for 2023-09-30
Replacing existing data for 2023-09-30
Replacing existing data for 2023-10-31
Replacing existing data for 2023-10-31
Replacing existing data for 2023-10-31
Replacing existing data for 2023-11-30
Replacing existing data for 2023-11-30
Replacing existing data for 2023-11-30
Replacing existing data for 2023-11-30
Replacing existing data for 2023-12-31
Replacing existing data for 2023-12-31
Replacing existing data for 2023-12-31
Replacing existing data for 2023-12-31
Replacing existing data for 2023-12-31
Replacing existing data f